<a href="https://colab.research.google.com/github/DaviAmerico/Solucao-Kaggle_Titanic_Machine_Learning_from_Disaster/blob/main/Competi%C3%A7%C3%A3o_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resolução da competição Titanic - Machine Learning from Disaster

Nosso objetivo aqui é criar um modelo que prevê se um dado passageiro com determinadas características sobreviverá ao naufrágio do Titanic , temos , portanto , um problema de classificação a ser resolvido.Essa resolução será divida em três tópicos : **Limpeza dos dados** , **Transformações** , **Criação do modelo**.

**Limpeza dos dados:**


Essa seção consiste em verificar se existe valores faltantes (*nan's*), e, se caso exista, substituir os *nan's* presentes nos dados , pois a presença deles acabam impossibilitando/prejudicando a criação de um modelo.

Antes disso vamos importar os dados para um Pandas DataFrame no Python e dar uma rápida visualizada nos mesmos:

In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
train=pd.read_csv('/content/drive/MyDrive/train1.csv')
test=pd.read_csv('/content/drive/MyDrive/test1.csv')
print(train.head(),'\n','\n','\n')
print(test.head())

   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns] 
 
 

   PassengerId  Pclass  ... Cabin Embarked
0          892       3  ...   NaN        Q
1          893       3  ...   NaN        S
2          894       2  ...   NaN        Q
3          895       3  ...   NaN        S
4          896       3  ...   NaN        S

[5 rows x 11 columns]


O conjunto *train* será o conjunto usado para ensinar ao modelo se um determinado passageiro sobreviverá, já o conjunto *test* é o conjunto de passageiros que queremos prever se sobreviverão ou não, portanto esse conjunto não contém a variável resposta, por esse motivo que ele contém uma coluna a menos que *train*. 

Como podemos ver há alguns valores faltantes em ambos os dados, agora vamos verificar quantos são em cada um:

In [81]:
###Total de NAN's no treino e teste:
print(train.isna().sum().sum())
print(test.isna().sum().sum())

866
414


Agora vamos substituir esses *nan's*, para os *nan's* de colunas númericas será atribuido a média da coluna e para os de colunas categóricas será atribuido o valor mais frequente da coluna, então:

In [82]:
###Substituindo NAN's numericos:
train.fillna(train.mean(),inplace=True)
test.fillna(test.mean(),inplace=True)
###Substituindo NAN's categoricos:
for i in train.select_dtypes('object').columns:##nao precisamos fazer um for para cada dado pq a resposta nao esta categorizada
    train.loc[:,i].fillna(train.loc[:,i].value_counts().index[0],inplace=True)
    test.loc[:,i].fillna(test.loc[:,i].value_counts().index[0],inplace=True)

Podemos verificar agora que não existem mais valores faltantes:

In [78]:
print(train.isna().sum().sum())
print(test.isna().sum().sum())

0
0


**Transformações:**


Nessa seção iremos iremos numerizar os dados , separar os preditores da variável resposta e depois normalizar os preditores. 

Transformando os dados em numéricos:

In [83]:
for i in train.select_dtypes('object').columns:##nao faz diferença pq a resposta nao esta categorizada
  train.loc[:,i]=LabelEncoder().fit_transform(train.loc[:,i].astype('str'))
  if i=='Name':
    a=LabelEncoder()##futuramente precisarei disso para saber quem irá sobreviver
    test.loc[:,i]=a.fit_transform(test.loc[:,i].astype('str'))
  else:
    test.loc[:,i]=LabelEncoder().fit_transform(test.loc[:,i].astype('str'))
##Agora podemos ver que cada atributo é numerico:
print(train,'\n','\n','\n')
print(test)

     PassengerId  Survived  Pclass  Name  ...  Ticket     Fare  Cabin  Embarked
0              1         0       3   108  ...     523   7.2500    145         2
1              2         1       1   190  ...     596  71.2833     81         0
2              3         1       3   353  ...     669   7.9250    145         2
3              4         1       1   272  ...      49  53.1000     55         2
4              5         0       3    15  ...     472   8.0500    145         2
..           ...       ...     ...   ...  ...     ...      ...    ...       ...
886          887         0       2   548  ...     101  13.0000    145         2
887          888         1       1   303  ...      14  30.0000     30         2
888          889         0       3   413  ...     675  23.4500    145         2
889          890         1       1    81  ...       8  30.0000     60         0
890          891         0       3   220  ...     466   7.7500    145         1

[891 rows x 12 columns] 
 
 

     Pass

Separando os preditores da resposta:

In [84]:
aux=train.columns[train.columns!='Survived']
xt=train.loc[:,aux]##Preditores no conjunto train
yt=train.loc[:,'Survived']##Resposta no conjunto train
xte=test##Preditores no conjunto test

Normalizando os preditores:

In [85]:
zt=pd.DataFrame(StandardScaler().fit_transform(xt))##Preditores normalizados no conjunto train
zt.columns=xt.columns
zte=pd.DataFrame(StandardScaler().fit_transform(xte))##Preditores normalizados no conjunto test
zte.columns=xte.columns
##Observe agora que os preditores nos conjuntos train e test estão normalizados:
print(zt.head(),'\n','\n','\n')
print(zte.head(),'\n','\n','\n')

   PassengerId    Pclass      Name  ...      Fare     Cabin  Embarked
0    -1.730108  0.827377 -1.310217  ... -0.502445  0.447397  0.585954
1    -1.726220 -1.566107 -0.991410  ...  0.786845 -1.365075 -1.942303
2    -1.722332  0.827377 -0.357685  ... -0.488854  0.447397  0.585954
3    -1.718444 -1.566107 -0.672604  ...  0.420730 -2.101392  0.585954
4    -1.714556  0.827377 -1.671790  ... -0.486337  0.447397  0.585954

[5 rows x 11 columns] 
 
 

   PassengerId    Pclass      Name  ...      Fare    Cabin  Embarked
0    -1.727912  0.873482 -0.020718  ... -0.498407 -0.35149 -0.470915
1    -1.719625  0.873482  1.611889  ... -0.513274 -0.35149  0.700767
2    -1.711337 -0.315819  0.501385  ... -0.465088 -0.35149 -0.470915
3    -1.703050  0.873482  1.653326  ... -0.483466 -0.35149  0.700767
4    -1.694763  0.873482 -0.252764  ... -0.418471 -0.35149  0.700767

[5 rows x 11 columns] 
 
 



Agora já podemos criar o nosso modelo.

**Criação do modelo:**


Aqui usaremos um modelo bagging de Ensemble Learning chamado RandomForest, esse é um modelo baseado em várias árvores de decisão treinadas em vários subconjuntos aleatórios dos dados , cada conjunto com um número diminuido de preditores , a previsão de uma RandomForest, no problema de classificação, será a previsão mais frequente nas árvores de decisão criadas.

Abaixo testaremos 25 combinações de hiper-parâmetros dentro da RandomForest e buscaremos a combinação com a melhor acurácia média dentro da validação cruzada com *10 folds*, o modelo usado será o com a melhor combinação de hiper-parâmetros.

In [88]:
rf=RandomForestClassifier()
params={'bootstrap': [True, False],
 'criterion':['entropy','gini'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000]}
f=RandomizedSearchCV(rf,params,cv=10,n_iter=25,scoring='accuracy')
m=f.fit(zt,yt)

Podemos ver a acurácia média do melhor modelo:

In [89]:
bs=f.best_score_
print(bs)

0.8529837702871411


Agora iremos prever quem sobreviverá ao naufrágio dentro do conjunto *test*:

In [90]:
yp=f.best_estimator_.predict(zte)
print(yp)

[0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 1 0 0 0 1 1 0 0 1
 1 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0
 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0
 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 1 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 1 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0
 1 1 0 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0
 1 0 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1
 0 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]


Segundo o nosso modelo os sobreviventes serão:

In [100]:
sobreviventes=((a.inverse_transform(test.loc[:,'Name']))[yp==1])
print(sobreviventes)

['Hirvonen, Mrs. Alexander (Helga E Lindqvist)'
 'Svensson, Mr. Johan Cervin' 'Connolly, Miss. Kate'
 'Caldwell, Mr. Albert Francis'
 'Abrahim, Mrs. Joseph (Sophie Halaut Easu)'
 'Snyder, Mrs. John Pillsbury (Nelle Stevenson)'
 'Chaffee, Mrs. Herbert Fuller (Carrie Constance Toogood)'
 'del Carlo, Mrs. Sebastiano (Argenia Genovesi)'
 'Ilmakangas, Miss. Ida Livija' 'Assaf Khalil, Mrs. Mariana (Miriam")"'
 'Flegenheim, Mrs. Alfred (Antoinette)'
 'Ryerson, Mrs. Arthur Larned (Emily Maria Borie)'
 'Ostby, Miss. Helene Ragnhild' 'Brady, Mr. John Bertram'
 'Dean, Mrs. Bertram (Eva Georgetta Light)'
 'Johnston, Mrs. Andrew G (Elizabeth Lily" Watson)"' 'Roth, Miss. Sarah A'
 'Cacic, Miss. Manda' 'Hee, Mr. Ling'
 'Corbett, Mrs. Walter H (Irene Colvin)'
 'Kimball, Mrs. Edwin Nelson Jr (Gertrude Parsons)'
 'Bucknell, Mrs. William Robert (Emma Eliza Ward)'
 'Coutts, Mrs. William (Winnie Minnie" Treanor)"'
 'Hocking, Miss. Ellen Nellie""' 'Fortune, Miss. Ethel Flora'
 'Chaudanson, Miss. Victorine' 

A proporção estimada de passageiros que sobreviverão é:

In [101]:
p=len(sobreviventes)/len(test)
print(p)

0.5382775119617225
